# Check what one data looks like

In [1]:
import xarray as xr
import pathlib

In [2]:
DATA_DIR = pathlib.Path("../21st_create_pdfmixs/pdfs_two_phase/")

In [3]:
def load_first_data(data_dir):
    fs = iter(data_dir.glob("*.nc"))
    return xr.load_dataset(next(fs))

In [4]:
ds = load_first_data(DATA_DIR)

In [5]:
ds

<xarray.Dataset>
Dimensions:    (r: 209, phase: 2)
Coordinates:
  * r          (r) float64 1.5 1.637 1.773 1.91 2.046 ... 29.5 29.64 29.77 29.91
  * phase      (phase) int32 0 1
Data variables: (12/15)
    G          (r) float64 -0.4412 -0.3948 0.3159 1.27 ... -2.367 1.495 3.558
    structure  (phase) object "# generated using pymatgen\ndata_MnP2O7\n_symm...
    fname      (phase) object 'mp-26982_MnP2O7' 'mp-1247607_SrCa7Mn6Al2O22'
    fraction   (phase) float64 0.5 0.5
    iso_adp    float64 0.008
    lat_scale  float64 1.0
    ...         ...
    qmin       float64 0.0
    qmax       float64 0.0
    qdamp      float64 0.0
    qbroad     float64 0.0
    delta1     float64 0.0
    delta2     float64 0.0

# Combine the data sets together

## Test the function

In [6]:
import xarray as xr
import tqdm
import typing

In [12]:
def concat_ncs(files: typing.Sequence[str], out_file: str) -> None:
    
    def gen_dss():
        dss = []
        for f in tqdm.tqdm(files):
            ds = xr.load_dataset(f)
            ds = ds[["G", "structure", "fraction"]]
            yield ds

    ds = xr.concat(gen_dss(), "mixture")
    ds.to_netcdf(out_file, engine="h5netcdf")
    ds.close()
    return

In [13]:
fs = iter(DATA_DIR.glob("*.nc"))

In [14]:
concat_ncs([next(fs), next(fs)], "./data/all_data_temp.nc")

100%|███████████████████████████████████████████████████| 2/2 [00:00<00:00, 60.44it/s]


In [15]:
xr.load_dataset("./data/all_data_temp.nc", engine="h5netcdf")

<xarray.Dataset>
Dimensions:    (mixture: 2, r: 209, phase: 2)
Coordinates:
  * phase      (phase) int32 0 1
  * r          (r) float64 1.5 1.637 1.773 1.91 2.046 ... 29.5 29.64 29.77 29.91
Dimensions without coordinates: mixture
Data variables:
    G          (mixture, r) float64 -0.4412 -0.3948 0.3159 ... -0.7657 -0.3646
    fraction   (mixture, phase) float64 0.5 0.5 0.3 0.7
    structure  (mixture, phase) object "# generated using pymatgen\ndata_MnP2...

## Run the function

In [17]:
ds = concat_ncs(DATA_DIR.glob("*.nc"), "./data/all_data.nc")

110000it [1:05:25, 28.02it/s]
